# Preprocessing Data

## Import Library

In [ ]:
!pip install scikit-learn-extra
!pip install geopy
from sklearn_extra.cluster import KMedoids
import pandas as pd
from geopy.geocoders import Bing
import numpy as np
import matplotlib.pyplot as plt
import time
import folium
import os
from tqdm import tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 7.9 MB/s eta 0:00:00


## Data Cleaning
Import Data Frame baru

In [ ]:
import pandas as pd
df = pd.read_csv('/content/hasil_eda.csv')

# Langkah Menangani Nilai yang Hilang
# Mengisi nilai yang hilang pada kolom 'tanggal' dengan metode forward fill
df['Tanggal Kejadian'].fillna(method='ffill', inplace=True)

# Mengisi nilai yang hilang pada kolom 'alamat' dengan string 'Unknown'
df['Alamat Kejadian'].fillna('Unknown', inplace=True)

# Mengisi nilai yang hilang pada kolom 'tanggal' dengan metode forward fill
df['Waktu Kejadian'].fillna(method='ffill', inplace=True)

# Mengubah kolom waktu_kejadian menjadi hanya waktu
df['Waktu Kejadian'] = pd.to_datetime(df['Waktu Kejadian']).dt.time

# Langkah Menghapus Duplikat
# Menghapus baris yang duplikat berdasarkan semua kolom
df.drop_duplicates(inplace=True)
df.head()

,Id,Tanggal Kejadian,Waktu Kejadian,Alamat Kejadian,Latitude,Longitude,Jenis Kendaraan R,jam,Tanggal,Bulan,Tahun,hari
0,1,13/01/2023,22:30:00,"JL. PH. H. MUSTOPA NO.31, KOTA BANDUNG KEL PAD...",NaN,NaN,2,22,13,1,2023,Jumat
1,2,21/01/2023,06:30:00,"JL. SIDOMUKTI NO 11, KEL. SUKALUYU, KEC. CIBEU...",NaN,NaN,2,6,21,1,2023,Sabtu
2,3,21/01/2023,06:30:00,"JL. SIDOMUKTI NO 11, KEL. SUKALUYU, KEC. CIBEU...",NaN,NaN,2,6,21,1,2023,Sabtu
3,4,21/01/2023,20:30:00,"JL. CIGADUNG RAYA TENGAH NO.28, KEL. CIGADUNG ...",NaN,NaN,2,20,21,1,2023,Sabtu
4,5,25/01/2023,23:00:00,"JL. CIGADUNG RAYA TIMUR, KEL. CIGADUNG KEC. CI...",NaN,NaN,4,23,25,1,2023,Rabu


In [ ]:
df.head()

,Id,Tanggal Kejadian,Waktu Kejadian,Alamat Kejadian,Latitude,Longitude,Jenis Kendaraan R,jam,Tanggal,Bulan,Tahun,hari
0,1,13/01/2023,22:30:00,"JL. PH. H. MUSTOPA NO.31, KOTA BANDUNG KEL PAD...",NaN,NaN,2,22,13,1,2023,Jumat
1,2,21/01/2023,06:30:00,"JL. SIDOMUKTI NO 11, KEL. SUKALUYU, KEC. CIBEU...",NaN,NaN,2,6,21,1,2023,Sabtu
2,3,21/01/2023,06:30:00,"JL. SIDOMUKTI NO 11, KEL. SUKALUYU, KEC. CIBEU...",NaN,NaN,2,6,21,1,2023,Sabtu
3,4,21/01/2023,20:30:00,"JL. CIGADUNG RAYA TENGAH NO.28, KEL. CIGADUNG ...",NaN,NaN,2,20,21,1,2023,Sabtu
4,5,25/01/2023,23:00:00,"JL. CIGADUNG RAYA TIMUR, KEL. CIGADUNG KEC. CI...",NaN,NaN,4,23,25,1,2023,Rabu


# Transform 'Waktu Kejadian' (time) to total minutes

## Detail data secara random

In [ ]:
df.iloc[452]

Id                                                                 453
Tanggal Kejadian                                            04/06/2021
Waktu Kejadian                                                21:23:00
Alamat Kejadian      Jl. Bojong Koneng, Sukapada, Kec. Cibeunying K...
Latitude                                                           NaN
Longitude                                                          NaN
Jenis Kendaraan R                                                    2
jam                                                                 21
Tanggal                                                              4
Bulan                                                                6
Tahun                                                             2021
hari                                                             Jumat
Name: 452, dtype: object

In [ ]:
df['Waktu Kejadian'] = pd.to_datetime(df['Waktu Kejadian'], format='%H:%M:%S').dt.hour * 60 + pd.to_datetime(df['Waktu Kejadian'], format='%H:%M:%S').dt.minute
df.head()

,Id,Tanggal Kejadian,Waktu Kejadian,Alamat Kejadian,Latitude,Longitude,Jenis Kendaraan R,jam,Tanggal,Bulan,Tahun,hari
0,1,13/01/2023,1350,"JL. PH. H. MUSTOPA NO.31, KOTA BANDUNG KEL PAD...",NaN,NaN,2,22,13,1,2023,Jumat
1,2,21/01/2023,390,"JL. SIDOMUKTI NO 11, KEL. SUKALUYU, KEC. CIBEU...",NaN,NaN,2,6,21,1,2023,Sabtu
2,3,21/01/2023,390,"JL. SIDOMUKTI NO 11, KEL. SUKALUYU, KEC. CIBEU...",NaN,NaN,2,6,21,1,2023,Sabtu
3,4,21/01/2023,1230,"JL. CIGADUNG RAYA TENGAH NO.28, KEL. CIGADUNG ...",NaN,NaN,2,20,21,1,2023,Sabtu
4,5,25/01/2023,1380,"JL. CIGADUNG RAYA TIMUR, KEL. CIGADUNG KEC. CI...",NaN,NaN,4,23,25,1,2023,Rabu


# Transform Jenis Kendaraan (2 => 0, 4 => 1)

In [ ]:
df['Jenis Kendaraan R'] = df['Jenis Kendaraan R'].replace({
    2: 0,
    4: 1
})

df.head()

,Id,Tanggal Kejadian,Waktu Kejadian,Alamat Kejadian,Latitude,Longitude,Jenis Kendaraan R,jam,Tanggal,Bulan,Tahun,hari
0,1,13/01/2023,1350,"JL. PH. H. MUSTOPA NO.31, KOTA BANDUNG KEL PAD...",NaN,NaN,0,22,13,1,2023,Jumat
1,2,21/01/2023,390,"JL. SIDOMUKTI NO 11, KEL. SUKALUYU, KEC. CIBEU...",NaN,NaN,0,6,21,1,2023,Sabtu
2,3,21/01/2023,390,"JL. SIDOMUKTI NO 11, KEL. SUKALUYU, KEC. CIBEU...",NaN,NaN,0,6,21,1,2023,Sabtu
3,4,21/01/2023,1230,"JL. CIGADUNG RAYA TENGAH NO.28, KEL. CIGADUNG ...",NaN,NaN,0,20,21,1,2023,Sabtu
4,5,25/01/2023,1380,"JL. CIGADUNG RAYA TIMUR, KEL. CIGADUNG KEC. CI...",NaN,NaN,1,23,25,1,2023,Rabu


# Adress to Coordinate Transform (Longitude, Latitude)

In [ ]:
df.head()

,Id,Tanggal Kejadian,Waktu Kejadian,Alamat Kejadian,Latitude,Longitude,Jenis Kendaraan R,jam,Tanggal,Bulan,Tahun,hari
0,1,13/01/2023,1350,"JL. PH. H. MUSTOPA NO.31, KOTA BANDUNG KEL PAD...",NaN,NaN,0,22,13,1,2023,Jumat
1,2,21/01/2023,390,"JL. SIDOMUKTI NO 11, KEL. SUKALUYU, KEC. CIBEU...",NaN,NaN,0,6,21,1,2023,Sabtu
2,3,21/01/2023,390,"JL. SIDOMUKTI NO 11, KEL. SUKALUYU, KEC. CIBEU...",NaN,NaN,0,6,21,1,2023,Sabtu
3,4,21/01/2023,1230,"JL. CIGADUNG RAYA TENGAH NO.28, KEL. CIGADUNG ...",NaN,NaN,0,20,21,1,2023,Sabtu
4,5,25/01/2023,1380,"JL. CIGADUNG RAYA TIMUR, KEL. CIGADUNG KEC. CI...",NaN,NaN,1,23,25,1,2023,Rabu


In [ ]:
# Import APIkey
geolocator2 = Bing("AoRWdZMkmnuS0FklLF2IuNA_tmjlTYnR7WG8TqD64xb5eNd-rcFtE70f7pSoNOJa")

# Atur ulang indeks setelah menghapus duplikat untuk menghindari KeyError
df.reset_index(drop=True, inplace=True)

for i in tqdm(range(len(df)), desc="Transformer"):
    address = df.loc[i, 'Alamat Kejadian']
    lokasi = geolocator2.geocode(address)
    df.loc[i, 'Latitude'] = lokasi.latitude
    df.loc[i, 'Longitude'] = lokasi.longitude
    time.sleep(1)

Transformer: 100%|██████████| 565/565 [11:49<00:00,  1.26s/it]


In [ ]:
df.tail()

,Id,Tanggal Kejadian,Waktu Kejadian,Alamat Kejadian,Latitude,Longitude,Jenis Kendaraan R,jam,Tanggal,Bulan,Tahun,hari
560,563,09/11/2021,360,"Jl. Pecah Kopi No.16, Sukaluyu, Kec. Cibeunyin...",-6.895886,107.633298,0,6,9,11,2021,Selasa
561,564,01/11/2021,915,"Jl. Cikutra Baru IV No.2, Neglasari, Kec. Cibe...",-6.895977,107.639263,0,15,1,11,2021,Senin
562,565,20/05/2021,735,"Jl. Cikutra Baru VII No.19, Neglasari, Kec. Ci...",-6.894716,107.639570,0,12,20,5,2021,Kamis
563,566,04/08/2021,660,"Jl. Cikutra Jl. Asrama GUPUSMU I No.201, Negla...",-6.894063,107.640153,1,11,4,8,2021,Rabu
564,567,04/11/2021,755,"Jl. Cikutra No.262, Neglasari, Kec. Cibeunying...",-6.893663,107.638752,0,12,4,11,2021,Kamis


# Check if there is null or NaN row

In [ ]:
df.isna().sum()

Id                   0
Tanggal Kejadian     0
Waktu Kejadian       0
Alamat Kejadian      0
Latitude             0
Longitude            0
Jenis Kendaraan R    0
jam                  0
Tanggal              0
Bulan                0
Tahun                0
hari                 0
dtype: int64

# Export to CSV New File

In [ ]:
# Ekspor dataFrame hasil preprocessing ke file CSV
#df.to_csv('hasil_preprocessing.csv', index=False)
#print("Data telah diekspor ke hasil_preprocessing.csv")

Data telah diekspor ke hasil_preprocessing.csv
